In [1]:
import argparse

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.model_selection import KFold

In [2]:
%load_ext autoreload
%autoreload 2

from load_data import load_data
from model import GModel
from myutils import roc_auc, translate_result
from optimizer import Optimizer
from sampler import Sampler

In [3]:
class Args:
    def __init__(self):
        self.device = "cuda"  # cuda:number or cpu
        self.data = "gdsc1"  # Dataset{gdsc or ccle}


args = Args()
res, drug_feature, exprs, mut, cna, null_mask, pos_num = load_data(args)

load gdsc1


In [4]:
epochs = []
true_datas = pd.DataFrame()
predict_datas = pd.DataFrame()
k = 5
kfold = KFold(n_splits=k, shuffle=True, random_state=42)
device = "cuda"

for i, (train_index, test_index) in enumerate(kfold.split(np.arange(pos_num))):
    sampler = Sampler(res, train_index, test_index, null_mask, i)
    model = GModel(
        adj_mat=sampler.train_data,
        gene=exprs,
        cna=cna,
        mutation=mut,
        sigma=2,
        k=11,
        iterates=3,
        feature_drug=drug_feature,
        n_hid1=192,
        n_hid2=36,
        alpha=5.74,
        device=device,
    )
    opt = Optimizer(
        model,
        sampler.train_data,
        sampler.test_data,
        sampler.test_mask,
        sampler.train_mask,
        roc_auc,
        lr=5e-4,
        epochs=1000,
        device=device,
    ).to(device)
    epoch, true_data, predict_data = opt()
    epochs.append(epoch)
    true_datas = pd.concat([true_datas, translate_result(true_data)])
    predict_datas = pd.concat([predict_datas, translate_result(predict_data)])

epoch:   0 loss:0.960009 auc:0.7583
epoch:  20 loss:0.293912 auc:0.9447
epoch:  40 loss:0.222333 auc:0.9478
epoch:  60 loss:0.202555 auc:0.9508
epoch:  80 loss:0.195184 auc:0.9529
epoch: 100 loss:0.190072 auc:0.9551
epoch: 120 loss:0.185262 auc:0.9573
epoch: 140 loss:0.182354 auc:0.9584
epoch: 160 loss:0.180821 auc:0.9590
epoch: 180 loss:0.179699 auc:0.9594
epoch: 200 loss:0.179219 auc:0.9598
epoch: 220 loss:0.178274 auc:0.9600
epoch: 240 loss:0.177626 auc:0.9602
epoch: 260 loss:0.176869 auc:0.9605
epoch: 280 loss:0.176538 auc:0.9607
epoch: 300 loss:0.175521 auc:0.9609
epoch: 320 loss:0.174426 auc:0.9612
epoch: 340 loss:0.174571 auc:0.9613
epoch: 360 loss:0.172840 auc:0.9616
epoch: 380 loss:0.171724 auc:0.9619
epoch: 400 loss:0.170990 auc:0.9623
epoch: 420 loss:0.171511 auc:0.9627
epoch: 440 loss:0.168980 auc:0.9633
epoch: 460 loss:0.166731 auc:0.9629
epoch: 480 loss:0.164467 auc:0.9647
epoch: 500 loss:0.181473 auc:0.9594
Fit finished.
epoch:   0 loss:0.837846 auc:0.4585
epoch:  20 los

In [5]:
true_datas.reset_index(drop=True).to_csv("true_gdsc1.csv")
predict_datas.reset_index(drop=True).to_csv("pred_gdsc1.csv")